<a href="https://colab.research.google.com/github/lolatelo/open-street-collisions/blob/main/pull_upload_to_bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install numpy

In [ ]:
import requests
import pandas as pd
from pandas_gbq import to_gbq
import json
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

def fetch_data(url, limit=50000):
    offset = 0
    data = []

    while True:
        # Update the URL with the current offset and limit
        paginated_url = f"{url}?$limit={limit}&$offset={offset}"
        response = requests.get(paginated_url)

        if response.status_code == 200:
            batch_data = response.json()

            # If no more data is returned, break out of the loop
            if not batch_data:
                break

            data.extend(batch_data)
            offset += limit
            print(f"Fetched {len(batch_data)} rows, total: {offset}")
        else:
            print("Failed to fetch data:", response.status_code)
            break

    return pd.DataFrame(data)

# Thoughts
- What about...a map that shows collisiosn, vs something else, and overlaps open streets on top?

# Motor Vehicle Collisons dataset


*   Contains details on crash events; each row is a crash event
*   The police report (MV104-AN) is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage
*   Other than homicides, the fatal incidents with which police have the most contact with the public are fatal traffic collisions
*   List item





In [5]:
# Example URL for a dataset API endpoint
url = "https://data.cityofnewyork.us/resource/h9gi-nx95.json"

# Make a GET request to fetch the data from the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data into a Python structure
    data = response.json()

    # Convert the data into a pandas DataFrame
    # Assuming `data` is a list of dictionaries
    df = pd.DataFrame(data)

    # Show the first few rows of the DataFrame
    print(df.head())
else:
    print("Failed to fetch data:", response.status_code)


                crash_date crash_time           on_street_name  \
0  2021-09-11T00:00:00.000       2:39    WHITESTONE EXPRESSWAY   
1  2022-03-26T00:00:00.000      11:45  QUEENSBORO BRIDGE UPPER   
2  2022-06-29T00:00:00.000       6:55       THROGS NECK BRIDGE   
3  2021-09-11T00:00:00.000       9:35                      NaN   
4  2021-12-14T00:00:00.000       8:13          SARATOGA AVENUE   

  off_street_name number_of_persons_injured number_of_persons_killed  \
0       20 AVENUE                         2                        0   
1             NaN                         1                        0   
2             NaN                         0                        0   
3             NaN                         0                        0   
4  DECATUR STREET                         0                        0   

  number_of_pedestrians_injured number_of_pedestrians_killed  \
0                             0                            0   
1                             0           

In [7]:
df.shape

(1000, 29)

In [8]:


# URL of the NYC Open Data API endpoint
api_url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json'

# Fetch the full dataset
df = fetch_data(api_url)

print(f"Total rows fetched: {len(df)}")


Fetched 50000 rows, total: 50000
Fetched 50000 rows, total: 100000
Fetched 50000 rows, total: 150000
Fetched 50000 rows, total: 200000
Fetched 50000 rows, total: 250000
Fetched 50000 rows, total: 300000
Fetched 50000 rows, total: 350000
Fetched 50000 rows, total: 400000
Fetched 50000 rows, total: 450000
Fetched 50000 rows, total: 500000
Fetched 50000 rows, total: 550000
Fetched 50000 rows, total: 600000
Fetched 50000 rows, total: 650000
Fetched 50000 rows, total: 700000
Fetched 50000 rows, total: 750000
Fetched 50000 rows, total: 800000
Fetched 50000 rows, total: 850000
Fetched 50000 rows, total: 900000
Fetched 50000 rows, total: 950000
Fetched 50000 rows, total: 1000000
Fetched 50000 rows, total: 1050000
Fetched 50000 rows, total: 1100000
Fetched 50000 rows, total: 1150000
Fetched 50000 rows, total: 1200000
Fetched 50000 rows, total: 1250000
Fetched 50000 rows, total: 1300000
Fetched 50000 rows, total: 1350000
Fetched 50000 rows, total: 1400000
Fetched 50000 rows, total: 1450000
Fetch

In [19]:
# Step 1: Create separate columns for latitude and longitude
# Check if the 'location' column exists and contains dictionaries
if 'location' in df.columns and isinstance(df.loc[0, 'location'], dict):
    # Extract latitude and longitude into their own columns
    df['latitude'] = df['location'].apply(lambda x: x.get('latitude') if isinstance(x, dict) else None)
    df['longitude'] = df['location'].apply(lambda x: x.get('longitude') if isinstance(x, dict) else None)

# Step 2: Delete the location column
df.drop('location', axis=1, inplace=True)

In [20]:
df.head()

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,zip_code,latitude,longitude,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2021-09-11T00:00:00.000,2:39,WHITESTONE EXPRESSWAY,20 AVENUE,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,QUEENSBORO BRIDGE UPPER,NaN,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-29T00:00:00.000,6:55,THROGS NECK BRIDGE,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-11T00:00:00.000,9:35,NaN,NaN,0,0,0,0,0,0,...,11208,40.667202,-73.8665,1211 LORING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-12-14T00:00:00.000,8:13,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,...,11233,40.683304,-73.917274,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Assuming you've already set up a GCP project and BigQuery dataset
project_id = 'open-data-2024'
dataset_id = 'nyc_motor_vechicle_crashes'
table_id = 'nyc_motor_vehicle_collisions'
table_full_name = f"{dataset_id}.{table_id}"

# Save the DataFrame to BigQuery
df.to_gbq(destination_table=table_full_name, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 281.76it/s]


# Open Streets

In [59]:
# Fetch
api_url = 'https://data.cityofnewyork.us/resource/uiay-nctu.json'
os_df = fetch_data(api_url)
print(f"Total rows fetched: {len(os_df)}")

Fetched 386 rows, total: 50000
Total rows fetched: 386


In [38]:
# Convert the 'the_geom' column to a string format
#os_df['the_geom'] = os_df['the_geom'].apply(lambda x: json.dumps(x))

In [60]:

# Extract the first set of coordinates (longitude1 and latitude1) and the second set (longitude2 and latitude2)
# Assuming that each MultiLineString has at least one line with at least two points
os_df['latitude1'] = os_df['the_geom'].apply(lambda x: x['coordinates'][0][0][1] if x['coordinates'] and len(x['coordinates'][0]) > 0 else None)
os_df['longitude1'] = os_df['the_geom'].apply(lambda x: x['coordinates'][0][0][0] if x['coordinates'] and len(x['coordinates'][0]) > 0 else None)
os_df['latitude2'] = os_df['the_geom'].apply(lambda x: x['coordinates'][0][1][1] if x['coordinates'] and len(x['coordinates'][0]) > 1 else None)
os_df['longitude2'] = os_df['the_geom'].apply(lambda x: x['coordinates'][0][1][0] if x['coordinates'] and len(x['coordinates'][0]) > 1 else None)

In [62]:
os_df.drop('the_geom', axis=1, inplace=True)


In [139]:
os_df.columns

Index(['object_id', 'orgname', 'appronstre', 'boroughname', 'apprfromst',
       'apprtostre', 'apprdayswe', 'reviewstat', 'apprmonope', 'apprmonclo',
       'apprtueope', 'apprtueclo', 'apprwedope', 'apprwedclo', 'apprthuope',
       'apprthuclo', 'apprfriope', 'apprfriclo', 'apprstartd', 'apprenddat',
       'shape_stle', 'segmentidt', 'segmentidf', 'apprsatope', 'apprsatclo',
       'apprsunope', 'apprsunclo', 'latitude1', 'longitude1', 'latitude2',
       'longitude2'],
      dtype='object')

In [63]:
# Assuming you've already set up a GCP project and BigQuery dataset
project_id = 'open-data-2024'
dataset_id = 'nyc_motor_vechicle_crashes'
table_id = 'open_streets'
table_full_name = f"{dataset_id}.{table_id}"

# Save the DataFrame to BigQuery
os_df.to_gbq(destination_table=table_full_name, project_id=project_id, if_exists='replace', progress_bar=True)

100%|██████████| 1/1 [00:00<00:00, 1495.30it/s]


# Analyzing

In [ ]:
# Assuming the data is stored in 'nyc_motor_vehicle_collisions.csv'
file_path = 'nyc_motor_vehicle_collisions.csv'

# Load the data
df = pd.read_csv(file_path)

# Initial inspection
print("First 5 rows of the dataset:")
print(df.head())

print("\nData types and missing values:")
print(df.info())

# Summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
print(df.describe())


In [65]:
print("\nSummary statistics for numerical columns:")
print(df.describe())


Summary statistics for numerical columns:
                     crash_date crash_time                    on_street_name  \
count                   2066119    2066119                           1628071   
unique                     4245       1440                             18368   
top     2014-01-21T00:00:00.000      16:00  BROADWAY                           
freq                       1161      28325                             17271   

                         off_street_name number_of_persons_injured  \
count                            1286368                   2066101   
unique                             20207                        32   
top     3 AVENUE                                                 0   
freq                                9837                   1595829   

       number_of_persons_killed number_of_pedestrians_injured  \
count                   2066088                       2066119   
unique                        7                            14   
top       

In [66]:
df.head()

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,zip_code,latitude,longitude,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,2021-09-11T00:00:00.000,2:39,WHITESTONE EXPRESSWAY,20 AVENUE,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-03-26T00:00:00.000,11:45,QUEENSBORO BRIDGE UPPER,NaN,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-29T00:00:00.000,6:55,THROGS NECK BRIDGE,NaN,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-09-11T00:00:00.000,9:35,NaN,NaN,0,0,0,0,0,0,...,11208,40.667202,-73.8665,1211 LORING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-12-14T00:00:00.000,8:13,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,...,11233,40.683304,-73.917274,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Libraries like geopandas for Python can be used for more complex spatial analyses and for preparing data for visualizations, which can then be plotted using mapping libraries or tools that support GeoJSON or similar geographic data formats

In [67]:
def is_on_open_street(longitude, latitude, os_df):
    for _, row in os_df.iterrows():
        if ((row['longitude1'] <= longitude <= row['longitude2']) or (row['longitude2'] <= longitude <= row['longitude1'])) and \
           ((row['latitude1'] <= latitude <= row['latitude2']) or (row['latitude2'] <= latitude <= row['latitude1'])):
            return True
    return False

In [68]:
df['on_open_street'] = df.apply(lambda x: is_on_open_street(x['longitude'], x['latitude'], os_df), axis=1)

TypeError: '<=' not supported between instances of 'float' and 'str'

## Geopandas mapping

In [69]:
import geopandas as gpd
from shapely.geometry import Point, LineString

# Convert df to a GeoDataFrame
gdf_collisions = gpd.GeoDataFrame(df, geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)], crs="EPSG:4326")

# Convert os_df to a GeoDataFrame
gdf_open_streets = gpd.GeoDataFrame(os_df, geometry=[LineString(xy) for xy in zip(zip(os_df.longitude1, os_df.latitude1), zip(os_df.longitude2, os_df.latitude2))], crs="EPSG:4326")

# Perform a spatial join to find collisions on open streets
# Note: Depending on the precision needed, you may adjust the operation type and tolerance
collisions_on_open_streets = gpd.sjoin(gdf_collisions, gdf_open_streets, how='left', op='intersects')

# Add a flag to indicate whether a collision occurred on an open street
collisions_on_open_streets['on_open_street'] = collisions_on_open_streets.index_right.notnull()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [ ]:
collisions_on_open_streets.head()

In [137]:
# Filter for collisions on '31ST AVE'
collisions_on_31st_ave = collisions_on_open_streets[
    (collisions_on_open_streets['on_street_name'].str.upper().str.contains('BERRY')) |
    (collisions_on_open_streets['off_street_name'].str.upper().str.contains('BERRY'))
]

# Display the filtered DataFrame
collisions_on_31st_ave

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,apprsatope,apprsatclo,apprsunope,apprsunclo,latitude1,longitude1,latitude2,longitude2,on_open_street,open_street_name
1693,2021-04-13T00:00:00.000,0:00,CANAL STREET,MULBERRY STREET,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4850,2021-04-22T00:00:00.000,17:50,BERRY STREET,NORTH 12 STREET,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
6770,2021-04-26T00:00:00.000,0:20,BERRY STREET,NORTH 11 STREET,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
10046,2021-09-09T00:00:00.000,0:40,ROSSVILLE AVENUE,BERRY COURT,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
12467,2021-09-20T00:00:00.000,10:30,CANAL STREET,MULBERRY STREET,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053847,2023-12-24T00:00:00.000,1:00,REMSEN STREET,NEWBERRY AVENUE,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2054306,2023-12-18T00:00:00.000,22:17,BERRY STREET,SOUTH 8 STREET,1,0,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2059017,2024-01-12T00:00:00.000,6:50,MULBERRY AVENUE,TRAVIS AVENUE,2,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2059558,2024-01-17T00:00:00.000,21:45,COLDEN STREET,MULBERRY AVENUE,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN


In [148]:
# prompt: find values in appronstreet field in os_df that are like 'berry street' or 'Berry Street' or 'BERRY STREET'

mask = os_df['appronstre'].str.contains('berry street', case=False, regex=True)
berry_streets = os_df[mask]
berry_streets.head()

,object_id,orgname,appronstre,boroughname,apprfromst,apprtostre,apprdayswe,reviewstat,apprmonope,apprmonclo,...,segmentidt,segmentidf,apprsatope,apprsatclo,apprsunope,apprsunclo,latitude1,longitude1,latitude2,longitude2
121,122,North Brooklyn Open Streets Community Coalitio...,BERRY STREET,Brooklyn,BROADWAY,NORTH 12 STREET,"mon,tue,wed,thu,fri,sat,sun",approvedLimited,08:00,20:00,...,0030447,0030447,08:00,20:00,08:00,20:00,40.711006,-73.965369,40.710482,-73.965492
122,123,North Brooklyn Open Streets Community Coalitio...,BERRY STREET,Brooklyn,BROADWAY,NORTH 12 STREET,"mon,tue,wed,thu,fri,sat,sun",approvedLimited,08:00,20:00,...,0030588,0030588,08:00,20:00,08:00,20:00,40.712459,-73.964463,40.711678,-73.964958
123,124,North Brooklyn Open Streets Community Coalitio...,BERRY STREET,Brooklyn,BROADWAY,NORTH 12 STREET,"mon,tue,wed,thu,fri,sat,sun",approvedLimited,08:00,20:00,...,0030614,0030614,08:00,20:00,08:00,20:00,40.713079,-73.964071,40.712459,-73.964463
124,125,North Brooklyn Open Streets Community Coalitio...,BERRY STREET,Brooklyn,BROADWAY,NORTH 12 STREET,"mon,tue,wed,thu,fri,sat,sun",approvedLimited,08:00,20:00,...,0030616,0030616,08:00,20:00,08:00,20:00,40.713826,-73.963601,40.713079,-73.964071
125,126,North Brooklyn Open Streets Community Coalitio...,BERRY STREET,Brooklyn,BROADWAY,NORTH 12 STREET,"mon,tue,wed,thu,fri,sat,sun",approvedLimited,08:00,20:00,...,0030618,0030618,08:00,20:00,08:00,20:00,40.714455,-73.963202,40.713826,-73.963601


In [150]:
mask = os_df['appronstre'].str.contains('31 ave', case=False, regex=True)
thiryfirststave_streets = os_df[mask]
thiryfirststave_streets

,object_id,orgname,appronstre,boroughname,apprfromst,apprtostre,apprdayswe,reviewstat,apprmonope,apprmonclo,...,segmentidt,segmentidf,apprsatope,apprsatclo,apprsunope,apprsunclo,latitude1,longitude1,latitude2,longitude2
243,244,31st Ave Open Street Collective,31 AVENUE,Queens,33 STREET,35 STREET,"sat,sun",approvedFull,NaN,NaN,...,0067781,0067781,12:00,20:00,12:00,20:00,40.763574,-73.921880,40.763199,-73.921081
244,245,31st Ave Open Street Collective,31 AVENUE,Queens,33 STREET,35 STREET,"sat,sun",approvedFull,NaN,NaN,...,0067896,0067896,12:00,20:00,12:00,20:00,40.763199,-73.921081,40.762823,-73.920283


In [83]:
# prompt: check values and counts of null latitude and longitude values in df
# Check for null values in latitude and longitude columns
null_lat = df['latitude'].isnull().value_counts()
null_lon = df['longitude'].isnull().value_counts()

# Print the results
print("Null values in latitude:")
print(null_lat)
print("\nNull values in longitude:")
print(null_lon)


Null values in latitude:
False    1832998
True      233121
Name: latitude, dtype: int64

Null values in longitude:
False    1832998
True      233121
Name: longitude, dtype: int64


In [155]:
df['latitude'].isnull().value_counts()/df['latitude'].shape[0]

False    0.88717
True     0.11283
Name: latitude, dtype: float64

In [74]:

# Ensure the open streets GeoDataFrame has a descriptive column (e.g., 'street_name')
# For demonstration, assuming 'street_name' exists in gdf_open_streets

# Map the 'street_name' from gdf_open_streets to collisions based on the index_right from the spatial join
collisions_on_open_streets['open_street_name'] = collisions_on_open_streets['index_right'].map(gdf_open_streets['appronstre'])

# Note: If 'index_right' is NaN for some rows, it means those collisions didn't occur on an open street
# and thus 'open_street_name' will be NaN for those rows as well


In [77]:
collisions_on_open_streets['on_open_street'].describe()

count     2066119
unique          1
top         False
freq      2066119
Name: on_open_street, dtype: object

In [78]:
# prompt: show the distinct values in the field 'on_open_street' from collisions_on_open_street and their associated counts

print(collisions_on_open_streets['on_open_street'].value_counts())

False    2066119
Name: on_open_street, dtype: int64


In [ ]:

# If you want to merge this information back into the original df DataFrame
# df['on_open_street'] = collisions_on_open_streets['on_open_street'].values


# Geomapping
Google maps was not free, so let's use Nomaintaim from OpenStreetMap

In [130]:
# 1. DataFrame with Existing Latitude and Longitude
df_with_coords = df.dropna(subset=['latitude', 'longitude'])

# 2. DataFrame Without Latitude and Longitude
df_without_coords = df[pd.isnull(df['latitude']) | pd.isnull(df['longitude'])]

In [136]:
df_with_coords.head()

,crash_date,crash_time,on_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,...,zip_code,latitude,longitude,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
3,2021-09-11T00:00:00.000,9:35,NaN,NaN,0,0,0,0,0,0,...,11208,40.667202,-73.8665,1211 LORING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-12-14T00:00:00.000,8:13,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,...,11233,40.683304,-73.917274,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-12-14T00:00:00.000,17:05,BROOKLYN QUEENS EXPRESSWAY,NaN,0,0,0,0,0,0,...,NaN,40.709183,-73.956825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-12-14T00:00:00.000,8:17,NaN,NaN,2,0,0,0,0,0,...,10475,40.86816,-73.83148,344 BAYCHESTER AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-12-14T00:00:00.000,21:10,NaN,NaN,0,0,0,0,0,0,...,11207,40.67172,-73.8971,2047 PITKIN AVENUE,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [129]:
# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# Optional: Use a rate limiter to avoid hitting service limits
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to geocode an address using Nominatim
def geocode_address(address):
    try:
        location = geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        print(f"Error geocoding {address}")
        return None, None

# Example usage
address = "1211 Loring Avenue, Brookyln, NY"
latitude, longitude = geocode_address(address)
print(f"Latitude: {latitude}, Longitude: {longitude}")


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/geopy/geocoders/base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "/usr/local/lib/python3.10/dist-packages/geopy/adapters.py", line 447, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "/usr/local/lib/python3.10/dist-packages/geopy/adapters.py", line 475, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 403

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/geopy/extra/rate_limiter.py", line 136, in _retries_gen
    yield i  # Run the function.
  File "/usr/local/lib/python3.10/dist-packages/geopy/extra/rate_limiter.py", line 274, in __call__
    res = self.func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/geopy/geocoders/nomin

Latitude: None, Longitude: None


In [99]:
# prompt: show columns in df

df.columns


Index(['crash_date', 'crash_time', 'on_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'cross_street_name',
       'contributing_factor_vehicle_3', 'vehicle_type_code_3',
       'contributing_factor_vehicle_4', 'vehicle_type_code_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_5'],
      dtype='object')

In [124]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Initialize the Nominatim geocoder with your specific User-Agent
geolocator = Nominatim(user_agent="geoapiExercises")

# Use a rate limiter to avoid hitting service limits
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Function to geocode an address using Nominatim
def geocode_street(row):
    on_street = row['on_street_name']
    off_street = row['off_street_name']
    borough = row['borough'] if pd.notnull(row['borough']) else "New York"

    # Constructing the address with on_street and off_street
    if pd.notnull(on_street) and pd.notnull(off_street):
        address = f"{on_street} & {off_street}, {borough}, NY"
    elif pd.notnull(on_street):
        address = f"{on_street}, {borough}, NY"
    elif pd.notnull(off_street):
        address = f"{off_street}, {borough}, NY"
    else:
        # If both streets are null, default to just the borough/New York
        address = f"{borough}, NY"

    try:
        location = geocode(address, timeout=1000)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return pd.Series([None, None])

# Assuming 'df' has columns named 'on_street_name' and 'borough'
# Apply the function to each row and create new columns for latitude and longitude


In [117]:
df_copy = df.loc[:10].copy()

In [125]:
# Apply the function to only the first 100 rows of the copied DataFrame
df_copy[['geocode_latitude', 'geocode_longitude']] = df_copy.apply(geocode_street, axis=1)

In [127]:
df_copy.loc[3]

crash_date                       2021-09-11T00:00:00.000
crash_time                                          9:35
on_street_name                                       NaN
off_street_name                                      NaN
number_of_persons_injured                              0
number_of_persons_killed                               0
number_of_pedestrians_injured                          0
number_of_pedestrians_killed                           0
number_of_cyclist_injured                              0
number_of_cyclist_killed                               0
number_of_motorist_injured                             0
number_of_motorist_killed                              0
contributing_factor_vehicle_1                Unspecified
contributing_factor_vehicle_2                        NaN
collision_id                                     4456314
vehicle_type_code1                                 Sedan
vehicle_type_code2                                   NaN
borough                        

# Next steps
- Make a clean notebook!
- We have in the os_df dataset: street and beginning and ending street; and lat and long for beg and end boundaries, but not main street
- We have in df a lat and long for the crash, when applicable. we need to find the remaining lat and long. for that, we have maybe the cross street and maybe borough (can be used for almost exact mapping), or the onstreet and maybe the offstreet
- Keep on going for 88% of the data for the model; revisit the missing data at another time